In [43]:
from IPython.display import display, Markdown

In [44]:
class Term:
    def __init__(self, name:str, type:str, value=None):
        self.name = name
        self.type = type
        self.value = None
        
        if type == "const":
            if value:   
                self.value = value
            else:
                self.value = name    
    def __repr__(self):
        return "{:}:{:}".format(self.name, self.type)
    
    def to_latex(self):
        return "{" + self.name + "}_{"+self.type+"}"

    def copy(self):
        return Term(self.name, self.type, self.value)


class Atom:
    def __init__(self, name: str, args: list[Term], isPositive = True ):
        self.name = name
        self.args = args
        self.isPositive = isPositive

    def __repr__(self):
        return "{:}{:}({:})".format('~' if not self.isPositive else '', self.name, ", ".join(
                [
                    "{:}".format(term) for term in self.args
                ]))
    
    def to_latex(self):
         return "{:}{:}({:})".format('\\not' if not self.isPositive else '', self.name, ", ".join(
                [
                    term.to_latex() for term in self.args
                ]))

    def copy(self):
        return Atom(self.name, [arg.copy() for arg in self.args], self.isPositive)


class Conjunct:
    def __init__(self, args: list[Atom]):
        self.args = args

    def __repr__(self):
        if len(self.args) == 0:
            return "Пусто"

        return " & ".join(
            [
                "{:}".format(atom) for atom in self.args
            ]
        )
    
    def to_latex(self):
        if len(self.args) == 0:
            return "\\oslash"

        return " \\wedge ".join(
            [
                atom.to_latex() for atom in self.args
            ]
        )
    

    def copy(self):
        return Conjunct([arg.copy() for arg in self.args])

class Implication:
    def __init__(self, left: Conjunct, right : Conjunct):
        self.left = left
        self.right = right

    def __repr__(self):
        return "{:} -> {:}".format(self.left, self.right)

    def to_latex(self):
        return "{:} \\rightarrow {:}".format(self.left.to_latex(), self.right.to_latex())

    def copy(self):
        return Implication(self.left.copy(), self.right.copy())

class Knowleadge:
    def __init__(self, data: Conjunct, rules: list[Implication]):
        self.data = data
        self.rules = rules

    def __repr__(self):
        return "\n".join(["{:}".format(i) for i in self.data.args + self.rules])

    def to_latex(self):
        return "\n\n".join(["${:}$".format(i.to_latex()) for i in self.data.args + self.rules])


    def copy(self):
        return Knowleadge(
            self.data.copy(),
              [rule.copy() for rule in self.rules]
        )


In [45]:
def parse_term(exp:str):
    exp = exp.strip()

    if ':' in exp:
        name, t = [e.strip() for e in exp.split(':')]
        return Term(name, t)
    

    return Term(exp, "const" if exp[0].capitalize() == exp[0] else "var")


def parse_atom(exp:str):
    exp = exp.strip()

    isPositive = exp[0] != '~'
        
    if not isPositive:
        exp = exp[1:].strip()

    
    atom_name, atom_args = exp.split('(')

    atom_name = atom_name.strip()
    atom_args = atom_args.split(')')[0].strip()

    atom_terms = [parse_term(term) for term in atom_args.split(',')]
    
    return Atom(atom_name, atom_terms, isPositive)

def parse_conjunct(exp:str):
    exp = exp.strip()
    atoms = [parse_atom(atom) for atom in exp.split("&")]
    
    return Conjunct(atoms)

def parse_implication(exp:str): 
    exp = exp.strip()
    left, right = exp.split("->")
    return Implication(parse_conjunct(left), parse_conjunct(right))

print(parse_implication('S(x4) & ~M(Kola) -> R(x4)'))


S(x4:var) & ~M(Kola:const) -> R(x4:var)


In [46]:
def add_substitution(substitions: map, sub:str, term: Term):
    for s in substitions:
        if substitions[s].name == sub:
            substitions[s] = term
    substitions[sub] = term

def UnificateAtoms(left:Atom, right:Atom):
    if left.name != right.name:
        return
    if len(left.args) != len(right.args):
        return
    
    substitions = {}

    for i in range(len(left.args)):
        leftTerm = left.args[i]
        rightTerm = right.args[i]

        if leftTerm.name == rightTerm.name:
            continue

        if leftTerm.type == "const" and rightTerm.type == "const":
            if leftTerm.value != rightTerm.value:
                return
        elif leftTerm.type == "var" and rightTerm.type == "const":
            add_substitution(substitions, leftTerm.name, rightTerm)
        elif leftTerm.type == "const" and rightTerm.type == "var":
            add_substitution(substitions, rightTerm.name, leftTerm)
        elif leftTerm.type == "var" and rightTerm.type == "var":
            if leftTerm.name != rightTerm.name:
                add_substitution(substitions, leftTerm.name, rightTerm)
        else:
            print("errr")
    return substitions   
                
        

In [47]:
def ShareSubstitutions(substitutions, atoms: list[Atom]):
    for atom in atoms:
        for term in atom.args:
            for sub in substitutions:
                if sub == term.name:
                    term.name = substitutions[sub].name
                    term.type = substitutions[sub].type
                    if substitutions[sub].type == "const":
                        term.value = substitutions[sub].value
                    else:
                        term.isLinked = True

def Concluse(data: Conjunct, implicat: Implication):
    # print(data.args, implicat)

    result = implicat.right.copy()

    implicatAtoms = implicat.left.copy()
    
    globalSubstitutions = {}
    
    counter = 1
    while counter:
        counter = 0
        for implicatAtom in implicatAtoms.args:
            for dataAtom in data.args:
                if implicatAtom.isPositive != dataAtom.isPositive:
                    continue

                # print("Try", implicatAtom, dataAtom)
                substitutions = UnificateAtoms(implicatAtom, dataAtom)
                if substitutions == None:
                    continue
                # print("Unificated", implicatAtom, dataAtom, "Sub", substitutions)

                for sub in substitutions:
                    add_substitution(globalSubstitutions, sub, substitutions[sub])
                
                implicatAtoms.args.remove(implicatAtom)

                ShareSubstitutions(substitutions, result.args + implicatAtoms.args)
                counter += 1
                break
            
    if len(implicatAtoms.args):
        # print("Not resolved", implicatAtoms.args)
        return None
    return result, globalSubstitutions


print(
    Concluse(
        parse_conjunct("O(N:const, M:const) & M(M:const) & A(W:const) & E(N:const, A:const)"),
        parse_implication("M(x1:var) & O(N:const, x1:var) -> S(W:const, x1:var, N:const)")
    )    
)


(S(W:const, M:const, N:const), {'x1': M:const})


In [48]:
def parse_knowleadge(exp: str):
    data = []
    rules = []

    for string in exp.splitlines():
        string = string.strip()

        if not string:
            continue

        try:
            impl = parse_implication(string)
            rules.append(impl)
        except:
            atom = parse_atom(string)
            data.append(atom)
    return Knowleadge(Conjunct(data), rules)
    

def get_knowleadge(): 
    return parse_knowleadge(
"""
O(N, M)
M(M)
A(W)
E(N, A)
                            
W(y) & A(x) & S(x,y,z) & H(z) -> C(x)
M(x_1) & O(N, x_1) -> S(W, x_1, N)
M(x_2) -> W(x_2)
E(x_3,A) -> H(x_3)
""")

Markdown(get_knowleadge().to_latex())

$O({N}_{const}, {M}_{const})$

$M({M}_{const})$

$A({W}_{const})$

$E({N}_{const}, {A}_{const})$

$W({y}_{var}) \wedge A({x}_{var}) \wedge S({x}_{var}, {y}_{var}, {z}_{var}) \wedge H({z}_{var}) \rightarrow C({x}_{var})$

$M({x_1}_{var}) \wedge O({N}_{const}, {x_1}_{var}) \rightarrow S({W}_{const}, {x_1}_{var}, {N}_{const})$

$M({x_2}_{var}) \rightarrow W({x_2}_{var})$

$E({x_3}_{var}, {A}_{const}) \rightarrow H({x_3}_{var})$

In [52]:

def substitution_to_latex(subs):
    return "\\{" + ",".join([
            sub + " / " + subs[sub].to_latex() for sub in subs
        ]
    ) +"\\}"
    
        

def KnowleadgeProve(knowleadge: Knowleadge, goal: Atom):
    goal = goal.copy()
    knowleadge = knowleadge.copy()
    
    openRules = knowleadge.rules
    

    def isApplied(atoms:list[Atom], goal: Atom):
        def equalTypes(left:Atom, right: Atom):
            if left.name != right.name:
                return False
            if len(left.args) != len(right.args):
                return False
            for i in range(len(left.args)):
                if left.args[i].type != right.args[i].type:
                    return False
            return True

        for atom in atoms:
            if atom.isPositive != goal.isPositive or not equalTypes(atom, goal):
                continue
            if UnificateAtoms(atom, goal) != None:
                return True
        return False
    
    if isApplied(knowleadge.data.args, goal):
        return True

    counter = 1
    while counter and openRules:
        counter = 0
        for rule in openRules:
            res = Concluse(knowleadge.data, rule)
            if res:
                display(Markdown("\n\n".join([
                    "База фактов: $" + knowleadge.data.to_latex() + "$",
                    "Правило: $" +  rule.to_latex() + "$",
                    "$\\Rightarrow$ $"  + res[0].to_latex() + '$ с подстановкой ' + "${:}$".format(substitution_to_latex(res[1]))
                ])))
                
                
                knowleadge.data.args += res[0].args
                if isApplied(res[0].args, goal):
                    return True
                
                openRules.remove(rule)
                counter += 1

    print(knowleadge)
    return False


display(Markdown(get_knowleadge().to_latex()))
print()
KnowleadgeProve(get_knowleadge(), parse_atom("C(W)"))


$O({N}_{const}, {M}_{const})$

$M({M}_{const})$

$A({W}_{const})$

$E({N}_{const}, {A}_{const})$

$W({y}_{var}) \wedge A({x}_{var}) \wedge S({x}_{var}, {y}_{var}, {z}_{var}) \wedge H({z}_{var}) \rightarrow C({x}_{var})$

$M({x_1}_{var}) \wedge O({N}_{const}, {x_1}_{var}) \rightarrow S({W}_{const}, {x_1}_{var}, {N}_{const})$

$M({x_2}_{var}) \rightarrow W({x_2}_{var})$

$E({x_3}_{var}, {A}_{const}) \rightarrow H({x_3}_{var})$

База фактов: $O({N}_{const}, {M}_{const}) \wedge M({M}_{const}) \wedge A({W}_{const}) \wedge E({N}_{const}, {A}_{const})$

Правило: $M({x_1}_{var}) \wedge O({N}_{const}, {x_1}_{var}) \rightarrow S({W}_{const}, {x_1}_{var}, {N}_{const})$

$\Rightarrow$ $S({W}_{const}, {M}_{const}, {N}_{const})$ с подстановкой $\{x_1 / {M}_{const}\}$

База фактов: $O({N}_{const}, {M}_{const}) \wedge M({M}_{const}) \wedge A({W}_{const}) \wedge E({N}_{const}, {A}_{const}) \wedge S({W}_{const}, {M}_{const}, {N}_{const})$

Правило: $E({x_3}_{var}, {A}_{const}) \rightarrow H({x_3}_{var})$

$\Rightarrow$ $H({N}_{const})$ с подстановкой $\{x_3 / {N}_{const}\}$

База фактов: $O({N}_{const}, {M}_{const}) \wedge M({M}_{const}) \wedge A({W}_{const}) \wedge E({N}_{const}, {A}_{const}) \wedge S({W}_{const}, {M}_{const}, {N}_{const}) \wedge H({N}_{const})$

Правило: $M({x_2}_{var}) \rightarrow W({x_2}_{var})$

$\Rightarrow$ $W({M}_{const})$ с подстановкой $\{x_2 / {M}_{const}\}$

База фактов: $O({N}_{const}, {M}_{const}) \wedge M({M}_{const}) \wedge A({W}_{const}) \wedge E({N}_{const}, {A}_{const}) \wedge S({W}_{const}, {M}_{const}, {N}_{const}) \wedge H({N}_{const}) \wedge W({M}_{const})$

Правило: $W({y}_{var}) \wedge A({x}_{var}) \wedge S({x}_{var}, {y}_{var}, {z}_{var}) \wedge H({z}_{var}) \rightarrow C({x}_{var})$

$\Rightarrow$ $C({W}_{const})$ с подстановкой $\{y / {M}_{const},x / {W}_{const},z / {N}_{const}\}$

True